In [ ]:
"skeleton_data\S001C001P001R001A001.skeleton"

In [3]:
import torch

mu = torch.tensor()
logvar = torch.tensor()

std = logvar.mul(0.1).exp()
print(std.shape)
std = torch.clamp(std, max = 100)
print(std.shape)
eps = torch.empty_like(std).normal_()
print(eps.shape)

eps.mul(std) + mu

torch.Size([256])
torch.Size([256])
torch.Size([256])


tensor([ 1.8063e+00,  2.0895e+00,  5.9736e-01,  1.8155e+00,  3.0108e-01,
         4.3935e-01,  3.3295e+00,  3.8801e-01, -3.0477e-02,  7.7179e-01,
         2.1462e-01,  1.5370e-02, -3.6588e-01, -7.6570e-01,  9.8845e-01,
         7.7133e-01,  3.6575e-01,  1.8198e-01, -4.3959e-01, -2.9773e-01,
        -4.1035e-02,  1.9726e-01,  7.0137e-01,  1.8711e+00,  2.3458e+00,
         6.1498e-01,  6.3956e-01,  1.1892e+00,  5.2219e-01,  3.0641e+00,
         2.1344e+00,  2.0818e-03,  9.2607e-01,  2.0819e+00,  2.1478e+00,
        -1.4458e-01,  1.1439e+00,  1.7609e+00,  1.2734e+00,  1.5568e+00,
        -1.3236e-01,  2.4181e+00,  1.3456e+00, -2.6920e-01,  1.0540e+00,
         2.8673e-01,  1.1979e+00,  2.3122e+00,  1.3570e+00, -2.6434e+00,
         7.0119e-01,  2.1686e+00,  1.7480e+00, -1.7179e+00,  1.6029e+00,
         2.3255e+00,  1.4212e+00,  9.8104e-01,  1.9362e+00,  5.2958e-01,
         5.3748e-01,  1.3583e+00, -1.1335e+00,  2.2510e+00,  3.1502e-01,
         2.5226e+00,  1.6503e+00, -3.2971e-01,  2.5

In [12]:
import torch.nn.functional as F
import torch

x = torch.tensor([[0.7, 0.2, 0.1, 0.5]])
target = torch.tensor([1, 0, 0, 0])
confidence = 0.9
logprobs = F.softmax(x)
print(logprobs)
print(target.unsqueeze(1).shape)
nll_loss = -logprobs.gather(dim = -1, index = target.unsqueeze(0))
print(nll_loss)
nll_loss = nll_loss.squeeze(1)
print(nll_loss)
smooth_loss = -logprobs.mean(dim = -1)
print(smooth_loss)
loss = confidence * nll_loss + 0.1 * smooth_loss
print(loss)

print(nll_loss)

tensor([[0.3362, 0.2039, 0.1845, 0.2753]])
torch.Size([4, 1])
tensor([[-0.2039, -0.3362, -0.3362, -0.3362]])
tensor([[-0.2039, -0.3362, -0.3362, -0.3362]])
tensor([-0.2500])
tensor([[-0.2085, -0.3276, -0.3276, -0.3276]])
tensor([[-0.2039, -0.3362, -0.3362, -0.3362]])


<ipython-input-12-d9f5ce4792ad>:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logprobs = F.softmax(x)


In [1]:
import copy as cp
import multiprocessing as mp
import numpy as np
import os
import os.path as osp
from mmcv import dump
from tqdm import tqdm

from smp import mrlines

eps = 1e-3

def parse_skeleton_file(ske_name, root = "skeleton_data"):
    ske_file = osp.join(root, ske_name + ".skeleton")

    lines = mrlines(ske_file)
    idx = 0
    num_frames = int(lines[0])
    num_joints = 25
    idx += 1

    body_data = dict()
    fidx = 0

    for f in range(num_frames):
        num_bodies = int(lines[idx])
        idx += 1
        if num_bodies == 0:
            continue
        for b in range(num_bodies):
            bodyID = int(lines[idx].split()[0])
            if bodyID not in body_data:
                kpt = []
                body_data[bodyID] = dict(kpt = kpt, start = fidx)
            idx += 1
            assert int(lines[idx]) == 25
            idx += 1
            joints = np.zeros((25, 3), dtype = np.float32)

            for j in range(num_joints):
                line = lines[idx].split()
                joints[j, :3] = np.array(line[:3], dtype=np.float32)
                idx += 1
            body_data[bodyID]['kpt'].append(joints)
        fidx += 1

    for k in body_data:
        body_data[k]['motion'] = np.sum(np.var(np.vstack(body_data[k]['kpt']), axis=0))
        body_data[k]['kpt'] = np.stack(body_data[k]['kpt'])

    assert idx == len(lines)
    return body_data


def spread_denoising(body_data_list):
    wh_ratio = 0.8
    spnoise_ratio = 0.69754

    def get_valid_frames(kpt):
        valid_frames = []
        for i in range(kpt.shape[0]):
            x, y = kpt[i, :, 0], kpt[i, :, 1]
            if (x.max() - x.min()) <= wh_ratio * (y.max() - y.min()):
                valid_frames.append(i)
        return valid_frames

    for item in body_data_list:
        valid_frames = get_valid_frames(item['kpt'])
        if len(valid_frames) == item['kpt'].shape[0]:
            item['flag'] = True
            continue
        ratio = len(valid_frames) / item['kpt'].shape[0]
        if 1 - ratio >= spnoise_ratio:
            item['flag'] = False
        else:
            item['flag'] = True
            item['motion'] = min(item['motion'],
                                 np.sum(np.var(item['kpt'][valid_frames].reshape(-1, 3), axis=0)))
    body_data_list = [item for item in body_data_list if item['flag']]
    assert len(body_data_list) >= 1
    _ = [item.pop('flag') for item in body_data_list]
    body_data_list.sort(key=lambda x: -x['motion'])
    return body_data_list


def non_zero(kpt):
    s = 0
    e = kpt.shape[1]
    while np.sum(np.abs(kpt[:, s])) < eps:
        s += 1
    while np.sum(np.abs(kpt[:, e - 1])) < eps:
        e -= 1
    return kpt[:, s: e]


def gen_keypoint_array(body_data):
    length_threshold = 11

    body_data = cp.deepcopy(list(body_data.values()))
    body_data.sort(key=lambda x: -x['motion'])
    if len(body_data) == 1:
        return body_data[0]['kpt'][None]
    else:
        body_data = [item for item in body_data if item['kpt'].shape[0] > length_threshold]
        if len(body_data) == 1:
            return body_data[0]['kpt'][None]
        body_data = spread_denoising(body_data)
        if len(body_data) == 1:
            return body_data[0]['kpt'][None]
        max_fidx = 0

        for item in body_data:
            max_fidx = max(max_fidx, item['start'] + item['kpt'].shape[0])
        keypoint = np.zeros((2, max_fidx, 25, 3), np.float32)

        s1, e1, s2, e2 = body_data[0]['start'], body_data[0]['start'] + body_data[0]['kpt'].shape[0], 0, 0
        keypoint[0, s1: e1] = body_data[0]['kpt']
        for item in body_data[1:]:
            s, e = item['start'], item['start'] + item['kpt'].shape[0]
            if max(s1, s) >= min(e1, e):
                keypoint[0, s: e] = item['kpt']
                s1, e1 = min(s, s1), max(e, e1)
            elif max(s2, s) >= min(e2, e):
                keypoint[1, s: e] = item['kpt']
                s2, e2 = min(s, s2), max(e, e2)

        keypoint = non_zero(keypoint)
        if np.sum(np.abs(keypoint[0, 0, 1])) < eps and np.sum(np.abs(keypoint[1, 0, 1])) > eps:
            keypoint = keypoint[::-1]
        return keypoint


root = 'skeleton_data'
skeleton_files = os.listdir(root)
names = [x.split('.')[0] for x in skeleton_files]
names.sort()
missing = mrlines('ntu120_missing.txt')
missing = set(missing)
names = [x for x in names if x not in missing]

extended = False
for name in names:
    if int(name.split('A')[-1]) > 60:
        extended = True
        print('NTURGB+D 120 skeleton files detected, will generate both ntu60_3danno.pkl and ntu120_3danno.pkl. ')
        break

if not extended:
    print('NTURGB+D 120 skeleton files not detected, will only generate ntu60_3danno.pkl. ')


def gen_anno(name):
    body_data = parse_skeleton_file(name, root)
    if len(body_data) == 0:
        return None
    keypoint = gen_keypoint_array(body_data).astype(np.float16)
    label = int(name.split('A')[-1]) - 1
    total_frames = keypoint.shape[1]
    return dict(frame_dir=name, label=label, keypoint=keypoint, total_frames=total_frames)


anno_dict = {}
num_process = 8

if num_process == 1:
    # Each annotations has 4 keys: frame_dir, label, keypoint, total_frames
    for name in tqdm(names):
        anno_dict[name] = gen_anno(name)
else:
    pool = mp.Pool(num_process)
    annotations = pool.map(gen_anno, names)
    pool.close()
    for anno in annotations:
        anno_dict[anno['frame_dir']] = anno

names = [x for x in names if anno_dict is not None]
training_subjects = [
    1, 2, 4, 5, 8, 9, 13, 14, 15, 16, 17, 18, 19, 25, 27, 28, 31, 34, 35,
    38, 45, 46, 47, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 70, 74, 78,
    80, 81, 82, 83, 84, 85, 86, 89, 91, 92, 93, 94, 95, 97, 98, 100, 103
]

if extended:
    xsub_train = [name for name in names if int(name.split('P')[1][:3]) in training_subjects]
    xsub_val = [name for name in names if int(name.split('P')[1][:3]) not in training_subjects]
    xset_train = [name for name in names if int(name.split('S')[1][:3]) % 2 == 0]
    xset_val = [name for name in names if int(name.split('S')[1][:3]) % 2 == 1]
    split = dict(xsub_train=xsub_train, xsub_val=xsub_val, xset_train=xset_train, xset_val=xset_val)
    annotations = [anno_dict[name] for name in names]
    dump(dict(split=split, annotations=annotations), 'ntu120_3danno.pkl')

names = [name for name in names if int(name.split('A')[-1]) <= 60]
xsub_train = [name for name in names if int(name.split('P')[1][:3]) in training_subjects]
xsub_val = [name for name in names if int(name.split('P')[1][:3]) not in training_subjects]
xview_train = [name for name in names if 'C001' not in name]
xview_val = [name for name in names if 'C001' in name]
split = dict(xsub_train=xsub_train, xsub_val=xsub_val, xview_train=xview_train, xview_val=xview_val)
annotations = [anno_dict[name] for name in names]
dump(dict(split=split, annotations=annotations), 'ntu60_3danno.pkl')

c:\Users\d4\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\d4\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\d4\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


NTURGB+D 120 skeleton files detected, will generate both ntu60_3danno.pkl and ntu120_3danno.pkl. 


## Graph, Adjacency Matrix 생성

In [17]:
import numpy as np

class Graph(): 
    # for NTU RGB+D, spatial strategy

    def __init__(self, max_hop = 1): # distance의 개수에 따라 adjacency matrix의 개수도 달라짐
        self.max_hop = max_hop # the number of distance between two edges which is I pay attention to look at
        self_loop = [(i, i) for i in range(25)] # self loop

        neighbor_list = [(1, 2), (2, 21), (3, 21), (4, 3), (5, 21),
                        (6, 5), (7, 6), (8, 7), (9, 21), (10, 9),
                        (11, 10), (12, 11), (13, 1), (14, 13), (15, 14),
                        (16, 15), (17, 1), (18, 17), (19, 18), (20, 19),
                        (22, 23), (23, 8), (24, 25), (25, 12)]
                        # connected edges list

        neighbor_list = [(i - 1, j - 1) for (i, j) in neighbor_list] # for indexing
        self.edge = self_loop + neighbor_list
        self.center = 21 - 1

        dis = np.zeros((25, 25))

        for i, j in self.edge:
            dis[i, j] = 1
            dis[j, i] = 1

        dis = np.stack([np.linalg.matrix_power(dis, d) for d in range(0, max_hop + 1)]) > 0
        # 총 3개의 dis
        mat = np.zeros((25, 25)) + np.inf
        # 
        for i in range(max_hop, -1, -1):
            mat[dis[i]] = i

        self.dis = mat
        self.get_adjacency_matrix()

    def __str__(self):
        return self.A

    def get_adjacency_matrix(self):
        valid_distance = range(self.max_hop + 1) # (0, 1, ..., max_hop)
        adjacency = np.zeros((25, 25))

        for dis in valid_distance:
            adjacency[self.dis == dis] = 1
            
        normalize_adjacency = normalize_digraph(adjacency)   

        A = []
        for dis in valid_distance:

            A_root = np.zeros((25, 25)) # adjacency matrix of root
            A_closer = np.zeros((25, 25)) # adjacency matrix of closer
            A_further = np.zeros((25, 25)) # adjacency matrix of further
            
            for i in range(25):
                for j in range(25):
                    if self.dis[j, i] == dis:
                        if self.dis[j, self.center] == self.dis[i, self.center]: # self.center = 1
                            A_root[j, i] = normalize_adjacency[j, i]
                        elif self.dis[j, self.center] > self.dis[i, self.center]:
                            A_closer[j, i] = normalize_adjacency[j, i]
                        else :
                            A_further[j, i] = normalize_adjacency[j, i]
            
            if dis == 0:
                A.append(A_root)
                
            else :
                A.append(A_root + A_closer)
                A.append(A_further)

        A = np.stack(A)
        self.A = A

def normalize_digraph(A):
    Dl = np.sum(A, 0) #
    for i in range(len(Dl)):
        for j in range(len(Dl)):
            if Dl[i] > 0:
                A[j, i] = A[j, i] / Dl[i]
    return A

In [40]:
graph = Graph(2)
print(graph.A.shape) # a_root, a_closer1, a_further1, a_closer2, a_further2

(5, 25, 25)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class Model(nn.Module):
    def __init__(self, graph):
        super(Model, self).__init__()
        
        self.graph = graph
        A = torch.tensor(self.graph.A, dtype = torch.float32, requires_grad = False)
        self.register_buffer("A", A)

        spatial_kernel_size = A.size(0)
        temporal_kernel_size = 9
        kernel_size = (temporal_kernel_size, spatial_kernel_size)
        self.data_bn = nn.BatchNorm1d(54)

        self.st_gcn_networks = nn.ModuleList((
            st_gcn(3, 64, kernel_size, 1, residual = False), # 1
            st_gcn(64, 64, kernel_size, 1), # 2
            st_gcn(64, 64, kernel_size, 1), # 3
            st_gcn(64, 64, kernel_size, 1), # 4
            st_gcn(64, 128, kernel_size, 2), # 5
            st_gcn(128, 128, kernel_size, 1), # 6
            st_gcn(128, 128, kernel_size, 1), # 7
            st_gcn(128, 256, kernel_size, 2), # 8
            st_gcn(256, 256, kernel_size, 1), # 9
            st_gcn(256, 256, kernel_size, 1), # 10
        ))

        self.edge_importance = nn.ParameterList([
            nn.Parameter(torch.ones(self.A.size())) for i in self.st_gcn_networks
        ])

        self.fcn = nn.Conv2d(256, 120, kernel_size = 1)
    
    def forward(self, x):
        B, C, T, V, M = x.size()
        x = x.permute(0, 4, 3, 1, 2).contiguous()
        x = x.view(B * M, V * C, T)
        x = self.data_bn(x)
        x = x.view(B, M, V, C, T)
        x = x.permute(0, 1, 3, 4, 2).contiguous()
        x = x.view(B * M, C, T, V)

        for graphconv, importance in zip(self.st_gcn_networks, self.edge_importance):
            x, A = graphconv(x, self.A * importance)

        x = F.avg_pool2d(x, x.size()[2:])

        x = x.view(B, M, -1, 1, 1).mean(dim = 1)
        x = self.fcn(x)
        x = x.view(x.size(0), -1)

        return x

class st_gcn(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride = 1, residual = True, dropout = 0):
        super(st_gcn, self).__init__()
        self.gcn_kernel_size = kernel_size[1]
        self.tcn_kernel_size = kernel_size[0]
        self.gcn = nn.Conv2d(in_channels, out_channels * self.gcn_kernel_size, 1)

        self.tcn = nn.Sequential(
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace = True),
            nn.Conv2d(out_channels, out_channels, (self.tcn_kernel_size, 1), padding = ((self.tcn_kernel_size - 1) // 2, 0)),
            nn.BatchNorm2d(out_channels),
            nn.Dropout(dropout, inplace = True))

        if residual != True:
            self.residual = lambda x : 0
        elif in_channels == out_channels :
            self.residual = lambda x : x
        else :
            self.residual = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=(1,1), stride = (stride, 1)),
                nn.BatchNorm2d(out_channels)
            ) 

    def forward(self, x, A):

        res = self.residual(x) 

        x = self.convnet(x)
        n, kc, t, v = x.size()
        x = x.view(n, self.kernel_size, kc // self.kernel_size, t, v) # 5 dim
        x = torch.einsum("nkctv, kvw -> nctw", (x, A))

        x = self.tcn(x) + res
        x = torch.relu(x)

        return x, A

## Train

In [32]:
from torchvision import transforms
from torch.utils.data import DataLoader
import time
import copy
import torch
import torchvision
import os
import numpy as np
from torch.optim import lr_scheduler
from tqdm import tqdm